<a href="https://colab.research.google.com/github/arnav0K/va/blob/main/Final_Federated_Learning_96_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving m2.xlsx to m2.xlsx


In [ ]:
!pip install tensorflow

!pip install pandas openpyxl
!pip install numpy
!pip install flwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.5/542.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.2
    Uninstalling typer-0.15.2:
      Successfully uninstalled typer-0.15.2
ERROR: pip's dependency resolver 

In [ ]:
!pip install -U "flwr[simulation]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 12.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from typing import Dict, List, Tuple
import flwr as fl

# Load and preprocess data
df = pd.read_excel("m2.xlsx")
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Simulate 5 clients (non-IID split)
num_clients = 5
client_data = np.array_split(df, num_clients)

# Function to convert DataFrame to batched TensorFlow Dataset
def create_dataset(client_df, batch_size=10000):
    X = client_df.drop(columns=["malware", "apk"]).values.astype(np.float32)
    y = client_df["malware"].values.astype(np.float32)
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    return dataset.shuffle(buffer_size=len(X)).batch(batch_size)


# Model architecture
def build_model() -> tf.keras.Model:
    return tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(df.shape[1]-2,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

# Flower client class
class MalwareClient(fl.client.NumPyClient):
    def __init__(self, client_df):
        self.client_df = client_df  # Store raw DataFrame, not a tf.data.Dataset
        self.model = build_model()
        self.model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

    def get_parameters(self, config) -> List[np.ndarray]:
        return self.model.get_weights()

    def fit(self, parameters, config) -> Tuple[List[np.ndarray], int, Dict]:
        self.model.set_weights(parameters)
        dataset = create_dataset(self.client_df)
        self.model.fit(dataset, epochs=10, verbose=0)
        return self.model.get_weights(), len(self.client_df), {}

    def evaluate(self, parameters, config) -> Tuple[float, int, Dict]:
        self.model.set_weights(parameters)
        dataset = create_dataset(self.client_df)
        loss, accuracy = self.model.evaluate(dataset, verbose=0)
        return float(loss), len(self.client_df), {"accuracy": float(accuracy)}

# Federated averaging strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=5,
    min_evaluate_clients=2,
    min_available_clients=5,
)

# Client creation function
def client_fn(cid: str) -> fl.client.Client:
    return MalwareClient(client_data[int(cid)])

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=num_clients,
    config=fl.server.ServerConfig(num_rounds=20),
    strategy=strategy,
)


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2025-04-20 06:20:15,201	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7997961831.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'object_store_memory': 3998980915.0}
INFO :      Optimize your si

History (loss, distributed):
	round 1: 0.16471775621175766
	round 2: 0.12429456412792206
	round 3: 0.10818100348114967
	round 4: 0.09858095273375511
	round 5: 0.09509078040719032
	round 6: 0.09027446061372757
	round 7: 0.08208156749606133
	round 8: 0.08175843209028244
	round 9: 0.07592560350894928
	round 10: 0.07446949928998947
	round 11: 0.07369010150432587
	round 12: 0.06964031234383583
	round 13: 0.06651078537106514
	round 14: 0.06433068588376045
	round 15: 0.06252042762935162
	round 16: 0.06375252641737461
	round 17: 0.06099930964410305
	round 18: 0.05863778479397297
	round 19: 0.06019970402121544
	round 20: 0.05609068460762501

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from typing import Dict, List, Tuple
import flwr as fl
from flwr.server.strategy import FedProx

# Load and preprocess data
df = pd.read_excel("m2.xlsx")
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle

# Split into global train/test (80/20)
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Create balanced non-IID clients (5 clients)
num_clients = 5
malware_data = train_df[train_df["malware"] == 1]
benign_data = train_df[train_df["malware"] == 0]

client_datasets = []
for i in range(num_clients):
    # Balanced sampling (50% malware, 50% benign)
    client_malware = malware_data.sample(n=len(malware_data)//num_clients, random_state=i)
    client_benign = benign_data.sample(n=len(benign_data)//num_clients, random_state=i)
    client = pd.concat([client_malware, client_benign])
    client_datasets.append(client)

# Create global test dataset
def create_dataset(client_df, batch_size=64):
    X = client_df.drop(columns=["malware", "apk"]).values.astype(np.float32)
    y = client_df["malware"].values.astype(np.float32)
    return tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

global_test_dataset = create_dataset(test_df)

# Enhanced model architecture
def build_model() -> tf.keras.Model:
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(train_df.shape[1]-2,)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Flower client with class balancing
class MalwareClient(fl.client.NumPyClient):
    def __init__(self, client_df):
        self.client_df = client_df
        self.model = build_model()

        # Dynamic class weighting
        class_counts = client_df["malware"].value_counts()
        total = class_counts.sum()
        self.class_weights = {0: total/(2*class_counts[0]), 1: total/(2*class_counts[1])}

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

    def get_parameters(self, config) -> List[np.ndarray]:
        return self.model.get_weights()

    def fit(self, parameters, config) -> Tuple[List[np.ndarray], int, Dict]:
        self.model.set_weights(parameters)
        dataset = create_dataset(self.client_df)
        self.model.fit(
            dataset,
            epochs=15,  # Increased epochs
            class_weight=self.class_weights,
            verbose=0
        )
        return self.model.get_weights(), len(self.client_df), {}

    def evaluate(self, parameters, config) -> Tuple[float, int, Dict]:
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(create_dataset(self.client_df), verbose=0)
        return float(loss), len(self.client_df), {"accuracy": float(accuracy)}

# Custom FedProx strategy with global evaluation
class CustomFedProx(FedProx):
    def aggregate_fit(self, server_round, results, failures):
        aggregated_weights = super().aggregate_fit(server_round, results, failures)

        if aggregated_weights is not None:
            # Evaluate global model on test set
            global_model = build_model()
            global_model.compile(loss='binary_crossentropy', metrics=['accuracy'])
            global_model.set_weights(aggregated_weights[0])
            loss, accuracy = global_model.evaluate(global_test_dataset, verbose=0)
            print(f"\n[Global Round {server_round}] Test Accuracy: {accuracy*100:.2f}%")

        return aggregated_weights

# Strategy configuration
strategy = CustomFedProx(
    proximal_mu=0.2,  # FedProx regularization
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=5,
    min_evaluate_clients=2,
    min_available_clients=5,
)

# Client factory
def client_fn(cid: str) -> fl.client.Client:
    return MalwareClient(client_datasets[int(cid)])

# Run simulation
hist = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=num_clients,
    config=fl.server.ServerConfig(num_rounds=20),
    strategy=strategy,
)

# Print final metrics
global_model = build_model()
global_model.compile(loss='binary_crossentropy', metrics=['accuracy'])
global_model.set_weights(hist.metrics_distributed['aggregated_weights'][-1])
final_loss, final_acc = global_model.evaluate(global_test_dataset, verbose=0)
print(f"\nFinal Model Performance:")
print(f"Test Loss: {final_loss:.4f}")
print(f"Test Accuracy: {final_acc*100:.2f}%")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2025-04-20 06:38:55,225	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7998615552.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3999307776.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_

RuntimeError: Simulation crashed.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from typing import Dict, List, Tuple
import flwr as fl

# Load and shuffle the dataset
df = pd.read_excel("m2.xlsx")
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into global train/test (80/20)
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Create balanced non-IID clients (5 clients, 50% malware + 50% benign)
num_clients = 5
malware_data = train_df[train_df["malware"] == 1]
benign_data = train_df[train_df["malware"] == 0]

client_data = []
for i in range(num_clients):
    client_malware = malware_data.sample(n=len(malware_data) // num_clients, random_state=i)
    client_benign = benign_data.sample(n=len(benign_data) // num_clients, random_state=i)
    client_df = pd.concat([client_malware, client_benign])
    client_data.append(client_df.sample(frac=1, random_state=i).reset_index(drop=True))

# Function to convert DataFrame to batched TensorFlow Dataset
def create_dataset(client_df, batch_size=64):
    X = client_df.drop(columns=["malware", "apk"]).values.astype(np.float32)
    y = client_df["malware"].values.astype(np.float32)
    return tf.data.Dataset.from_tensor_slices((X, y)).shuffle(len(X)).batch(batch_size)

# Global test dataset
global_test_dataset = create_dataset(test_df)

# Model architecture
def build_model() -> tf.keras.Model:
    return tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(df.shape[1]-2,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

# Flower client class
class MalwareClient(fl.client.NumPyClient):
    def __init__(self, client_df):
        self.client_df = client_df
        self.model = build_model()
        self.model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

    def get_parameters(self, config) -> List[np.ndarray]:
        return self.model.get_weights()

    def fit(self, parameters, config) -> Tuple[List[np.ndarray], int, Dict]:
        self.model.set_weights(parameters)
        dataset = create_dataset(self.client_df)
        self.model.fit(dataset, epochs=10, verbose=0)
        return self.model.get_weights(), len(self.client_df), {}

    def evaluate(self, parameters, config) -> Tuple[float, int, Dict]:
        self.model.set_weights(parameters)
        dataset = create_dataset(self.client_df)
        loss, accuracy = self.model.evaluate(dataset, verbose=0)
        return float(loss), len(self.client_df), {"accuracy": float(accuracy)}

# Federated averaging strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=5,
    min_evaluate_clients=2,
    min_available_clients=5,
)

# Client creation function
def client_fn(cid: str) -> fl.client.Client:
    return MalwareClient(client_data[int(cid)])

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=num_clients,
    config=fl.server.ServerConfig(num_rounds=20),
    strategy=strategy,
)


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2025-04-20 07:19:46,714	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3998313676.0, 'node:172.28.0.12': 1.0, 'memory': 7996627355.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_

History (loss, distributed):
	round 1: 0.08293548598885536
	round 2: 0.06262973882257938
	round 3: 0.05663745291531086
	round 4: 0.0507198516279459
	round 5: 0.050710149109363556
	round 6: 0.048437800258398056
	round 7: 0.045054519549012184
	round 8: 0.04533318430185318
	round 9: 0.043943773955106735
	round 10: 0.03858521953225136
	round 11: 0.03746944107115269
	round 12: 0.039788445457816124
	round 13: 0.038315048441290855
	round 14: 0.036378199234604836
	round 15: 0.035859378054738045
	round 16: 0.0392959900200367
	round 17: 0.04017489776015282
	round 18: 0.038671283051371574
	round 19: 0.04291930980980396
	round 20: 0.043223921209573746

In [ ]:
def convert_loss_to_accuracy(losses):
    """
    Converts a list of loss values into accuracy percentages using accuracy = (1 - loss) * 100.

    Args:
        losses (list of float): List of loss values per round.

    Returns:
        list of float: List of accuracy percentages corresponding to each round.
    """
    accuracies = []
    for i, loss in enumerate(losses):
        accuracy = (1 - loss) * 100  # Convert to percentage
        accuracies.append(accuracy)
        print(f"Round {i+1}: Loss = {loss:.6f}, Accuracy = {accuracy:.2f}%")
    return accuracies

# Example usage:
loss_values = [
    0.08293548598885536, 0.06262973882257938, 0.05663745291531086,
    0.0507198516279459, 0.050710149109363556, 0.048437800258398056,
    0.045054519549012184, 0.04533318430185318, 0.043943773955106735,
    0.03858521953225136, 0.03746944107115269, 0.039788445457816124,
    0.038315048441290855, 0.036378199234604836, 0.035859378054738045,

]

# Call the function
accuracies = convert_loss_to_accuracy(loss_values)


Round 1: Loss = 0.082935, Accuracy = 91.71%
Round 2: Loss = 0.062630, Accuracy = 93.74%
Round 3: Loss = 0.056637, Accuracy = 94.34%
Round 4: Loss = 0.050720, Accuracy = 94.93%
Round 5: Loss = 0.050710, Accuracy = 94.93%
Round 6: Loss = 0.048438, Accuracy = 95.16%
Round 7: Loss = 0.045055, Accuracy = 95.49%
Round 8: Loss = 0.045333, Accuracy = 95.47%
Round 9: Loss = 0.043944, Accuracy = 95.61%
Round 10: Loss = 0.038585, Accuracy = 96.14%
Round 11: Loss = 0.037469, Accuracy = 96.25%
Round 12: Loss = 0.039788, Accuracy = 96.02%
Round 13: Loss = 0.038315, Accuracy = 96.17%
Round 14: Loss = 0.036378, Accuracy = 96.36%
Round 15: Loss = 0.035859, Accuracy = 96.41%
